In [2]:
%load_ext autoreload
%autoreload 2
%pylab inline
import pandas as pd
import os

Populating the interactive namespace from numpy and matplotlib


In [3]:
import pprint
pp = pprint.PrettyPrinter(indent=4)

In [4]:
import sys
sys.path.append("/home/muncok/DL/projects/sv_system/")

In [41]:
import csv
def write_manifest(dataset, save_path, label_idx):
    samples = []
    with open(save_path, 'w') as f:
        for index, row in dataset.iterrows():
            file_path = os.path.join(data_dir, row.dir, row.file)
            label = label_idx.index(row.spk)
            sample = ','.join([file_path, str(label)])
            samples.append(sample)
        random.shuffle(samples)
        writer = csv.writer(f, delimiter='\n', quoting=csv.QUOTE_NONE)
        writer.writerow(samples)
        print("{} was written".format(save_path))

# Dataframes
command: disjoint speaker split

In [36]:
dataframe_dir = '/home/muncok/DL/dataset/SV_sets/dataframes/'
data_dir = '/home/muncok/DL/dataset/SV_sets/speech_commands/'
data_df = pd.read_pickle(os.path.join(dataframe_dir,'Command_Dataframe.pkl'))
data_df['dir'] = data_df.sent

### Dataset split

In [37]:
#TODO: correct splits
all_spks = data_df.spk.unique()
all_sents = data_df.sent.unique()
uttrs_counts = data_df.spk.value_counts()
sv_spks = list(uttrs_counts.index[200:])
si_spks = list(uttrs_counts.index[:200])

In [38]:
train_spks = si_spks[:100]
val_spks = si_spks[100:150]

In [46]:
# random sampling
si_df = data_df[data_df.spk.isin(si_spks)]
train_sets = []
trainval_sets = []
val_sets = []
for spk in train_spks:
    train_set = si_df[si_df.spk==spk].sample(n=80)
    train_sets.append(train_set)
    trainval_sets.append(si_df[si_df.spk==spk].drop(index=train_set.index))
for spk in val_spks:
    val_sets.append(si_df[si_df.spk==spk].sample(n=80))

si_train = pd.concat(train_sets)
si_trainval = pd.concat(trainval_sets)
si_val = pd.concat(val_sets)
print("[random] train:{}, val:{}".format(len(si_train), len(si_val)))

[random] train:8000, val:4000


In [48]:
write_manifest(si_train, "manifest/commands/si_commands_train_manifest.csv", train_spks)

manifest/commands/si_commands_train_manifest.csv was written


In [51]:
write_manifest(si_trainval, "manifest/commands/si_commands_trainval_manifest.csv", train_spks)

manifest/commands/si_commands_trainval_manifest.csv was written


In [52]:
write_manifest(si_val, "manifest/commands/si_commands_val_manifest.csv", val_spks)
write_manifest(si_val, "manifest/commands/si_commands_test_manifest.csv", val_spks)

manifest/commands/si_commands_val_manifest.csv was written
manifest/commands/si_commands_test_manifest.csv was written


In [ ]:
base_train = si_train

## Baseline (Speaker Dicriminant DNN)

In [65]:
import honk_sv.train as train
import honk_sv.dataset as dset


global_config = dict(model=model, dataset="commands",
                     no_cuda=False,  gpu_no=0,
                     n_epochs=100, batch_size=64,
                     lr=[0.01, 0.001], schedule=[np.inf], dev_every=1, seed=0, use_nesterov=False,
                     cache_size=32768, momentum=0.9, weight_decay=0.00001,
                     num_workers=32, print_step=100,
                     splice_length=20, bn_size=256
                     )

builder = train.ConfigBuilder(dset.SpeechDataset.default_config(),
                             global_config)
parser = builder.build_argparse()
config = builder.config_from_argparse(parser)
config['data_dir'] = "/home/muncok/DL/dataset/SV_sets/speech_commands/"

In [85]:
import torch.nn as nn
x_dim = [1, 28, 28]
hid_dim = 64
z_dim = 64

class Flatten(nn.Module):
    def __init__(self):
        super(Flatten, self).__init__()

    def forward(self, x):
        return x.view(x.size(0), -1)

def conv_block(in_channels, out_channels):
    return nn.Sequential(
        nn.Conv2d(in_channels, out_channels, 3, padding=1),
        nn.BatchNorm2d(out_channels),
        nn.ReLU(),
        nn.MaxPool2d(2)
    )

model = nn.Sequential(Ordered
    conv_block(x_dim[0], hid_dim),
    conv_block(hid_dim, hid_dim),
    conv_block(hid_dim, hid_dim),
    conv_block(hid_dim, z_dim),
    Flatten(),
    nn.Linear(768, 100)
)

In [87]:
manifest_dir = "manifest/commands/"
config['train_manifest']=os.path.join(manifest_dir,'si_{}_{}_manifest.csv'.format("commands", "train"))
config['val_manifest']=os.path.join(manifest_dir,'si_{}_{}_manifest.csv'.format("commands", "trainval"))
config['test_manifest']=os.path.join(manifest_dir,"si_commands_trainval_manifest.csv")
config['model'] = "cnn"
config['n_labels'] = 100
config['output_file'] = "base_model.pth"
train.train(config, model=model)

train step #99 accuracy: 0.640625, loss: 1.5126124620437622
epoch #0, final dev accuracy: 0.6308962264150944
saving best model...
train step #199 accuracy: 0.765625, loss: 0.8648862242698669
epoch #1, final dev accuracy: 0.7920597484276729
saving best model...
train step #299 accuracy: 0.921875, loss: 0.4000552296638489
epoch #2, final dev accuracy: 0.7896029874213837
train step #399 accuracy: 0.859375, loss: 0.5234934687614441
train step #499 accuracy: 0.921875, loss: 0.3242689073085785
epoch #3, final dev accuracy: 0.8474842767295597
saving best model...
train step #599 accuracy: 0.953125, loss: 0.203951895236969
epoch #4, final dev accuracy: 0.8583922955974842
saving best model...
train step #699 accuracy: 0.96875, loss: 0.20342430472373962
epoch #5, final dev accuracy: 0.8835495283018868
saving best model...
train step #799 accuracy: 0.96875, loss: 0.14396411180496216
epoch #6, final dev accuracy: 0.8806014150943396
train step #899 accuracy: 0.953125, loss: 0.18563376367092133
trai

train step #7899 accuracy: 1.0, loss: 0.001346677541732788
train step #7999 accuracy: 1.0, loss: 0.0010441839694976807
epoch #63, final dev accuracy: 0.9637382075471698
saving best model...
train step #8099 accuracy: 1.0, loss: 0.0018312633037567139
epoch #64, final dev accuracy: 0.9515526729559748
train step #8199 accuracy: 1.0, loss: 0.0013029277324676514
epoch #65, final dev accuracy: 0.9604952830188679
train step #8299 accuracy: 1.0, loss: 0.0011591315269470215
epoch #66, final dev accuracy: 0.9550904088050314
train step #8399 accuracy: 1.0, loss: 0.0005782246589660645
train step #8499 accuracy: 1.0, loss: 0.0012674331665039062
epoch #67, final dev accuracy: 0.9604952830188679
train step #8599 accuracy: 1.0, loss: 0.0013405382633209229
epoch #68, final dev accuracy: 0.9547955974842767
train step #8699 accuracy: 1.0, loss: 0.0020675361156463623
epoch #69, final dev accuracy: 0.9610849056603774
train step #8799 accuracy: 1.0, loss: 0.002141624689102173
epoch #70, final dev accuracy: 

In [92]:
import torch
state_dict= torch.load("base_model.pth")

In [96]:
model.state_dict().update(state_dict)

In [103]:
model.parameters()

<generator object Module.parameters at 0x7fa7c5f73620>